In [1]:
from google.colab import drive

In [2]:
import os
import zipfile
from zipfile import ZipFile
from os import path
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, to_timestamp, unix_timestamp,mean, to_date, month,  date_format, array, explode, rank, months_between, year, max, min, row_number
from pyspark.sql import types
from pyspark.sql.window import Window

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls "/content/drive/MyDrive/data"

Divvy_Trips_2019_Q4.zip  Divvy_Trips_2020_Q1.zip


In [5]:
def get_file_paths(root_directory, folder_name, file_extension):
    file_paths = []
    file_directory = os.path.join(root_directory, folder_name)
    for root, _, files in os.walk(file_directory):
        for file_name in files:
            if file_extension in file_name:
                file_paths.append(os.path.join(root, file_name))
    return file_paths

In [6]:
def read_zip_files_into_dataframes(zip_file_paths):
    df_list = []
    for file_path in zip_file_paths:
        print(f"Reading for {file_path}")
        zip_file = ZipFile(file_path)
        for text_file in zip_file.infolist():
            if text_file.filename.endswith(
                ".csv"
            ) and not text_file.filename.startswith("__MACOSX"):
                df = pd.read_csv(zip_file.open(text_file.filename))
                # add a column for the year based on the filename
                year = int(path.basename(file_path).split(".")[0].split("_")[-2])
                df['year'] = year
                df_list.append(df)
    return df_list

In [7]:
def write_to_csv(df, file_path):
  # set mode to overwrite
  df.write.mode("overwrite").csv(file_path, header=True)

In [8]:
current_directory = os.getcwd()

In [9]:
current_directory

'/content'

In [10]:
zip_file_paths = get_file_paths(current_directory, r"/content/drive/MyDrive/data", "zip")

In [11]:
zip_file_paths

['/content/drive/MyDrive/data/Divvy_Trips_2019_Q4.zip',
 '/content/drive/MyDrive/data/Divvy_Trips_2020_Q1.zip']

In [12]:
pdf = read_zip_files_into_dataframes(zip_file_paths)

Reading for /content/drive/MyDrive/data/Divvy_Trips_2019_Q4.zip
Reading for /content/drive/MyDrive/data/Divvy_Trips_2020_Q1.zip


In [13]:
df_2019 = pdf[0]

In [14]:
df_2019['birthyear'] = df_2019['birthyear'].astype('Int64')
df_2019["trip_id"] = df_2019["trip_id"].astype(str)

In [15]:
df_2019=df_2019.rename(columns={'trip_id' : "ride_id", "start_time": "started_at", "end_time": "ended_at", "from_station_id": "start_station_id", "from_station_name": "start_station_name", "to_station_id": "end_station_id", "to_station_name": "end_station_name"})

In [ ]:
df_2020 = pdf[1]

In [ ]:
overlapping_columns = df_2020.columns.intersection(df_2019.columns).tolist()

In [ ]:
df_2019_subset = df_2019.loc[:, overlapping_columns]

In [ ]:
df_2020_subset = df_2020.loc[:, overlapping_columns]

In [ ]:
subset_list = [df_2019_subset, df_2020_subset]

In [ ]:
combined_df = pd.concat(subset_list, ignore_index=True)

In [ ]:
combined_df['end_station_id'] = combined_df['end_station_id'].astype('Int64')
combined_df['start_station_id'] = combined_df['start_station_id'].astype('Int64')

In [ ]:
cols = combined_df.columns.tolist()

In [ ]:
combined_df = combined_df[['ride_id','started_at', 'ended_at','start_station_id','start_station_name','end_station_id','end_station_name','year']]

In [ ]:

combined_df['started_at']= pd.to_datetime(combined_df['started_at'])
combined_df['ended_at'] = pd.to_datetime(combined_df['ended_at'])

In [ ]:
combined_df.fillna({'end_station_id': 0}, inplace=True)

In [ ]:
combined_df.dtypes

,0
ride_id,object
started_at,datetime64[ns]
ended_at,datetime64[ns]
start_station_id,Int64
start_station_name,object
end_station_id,Int64
end_station_name,object
year,int64


In [ ]:
schema = types.StructType([
    types.StructField("ride_id", types.StringType(), True),
    types.StructField("started_at", types.TimestampType(), True),
    types.StructField("ended_at", types.TimestampType(), True),
    types.StructField("start_station_id", types.LongType(), True),
    types.StructField("start_station_name", types.StringType(), True),
    types.StructField("end_station_id", types.LongType(), True),
    types.StructField("end_station_name", types.StringType(), True),
    types.StructField("year", types.LongType(), True),

])

In [20]:
spark = SparkSession.builder.appName("Exercise6").enableHiveSupport().getOrCreate()

In [21]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

In [ ]:
spark.createDataFrame(combined_df).schema

StructType([StructField('ride_id', StringType(), True), StructField('started_at', TimestampType(), True), StructField('ended_at', TimestampType(), True), StructField('start_station_id', LongType(), True), StructField('start_station_name', StringType(), True), StructField('end_station_id', LongType(), True), StructField('end_station_name', StringType(), True), StructField('year', LongType(), True)])

In [ ]:
sparkDF = spark.createDataFrame(combined_df, schema = schema)

In [ ]:
sparkDF.show()

+--------+-------------------+-------------------+----------------+--------------------+--------------+--------------------+----+
| ride_id|         started_at|           ended_at|start_station_id|  start_station_name|end_station_id|    end_station_name|year|
+--------+-------------------+-------------------+----------------+--------------------+--------------+--------------------+----+
|25223640|2019-10-01 00:01:39|2019-10-01 00:17:20|              20|Sheffield Ave & K...|           309|Leavitt St & Armi...|2019|
|25223641|2019-10-01 00:02:16|2019-10-01 00:06:34|              19|Throop (Loomis) S...|           241| Morgan St & Polk St|2019|
|25223642|2019-10-01 00:04:32|2019-10-01 00:18:43|              84|Milwaukee Ave & G...|           199|Wabash Ave & Gran...|2019|
|25223643|2019-10-01 00:04:32|2019-10-01 00:43:43|             313|Lakeview Ave & Fu...|           290|Kedzie Ave & Palm...|2019|
|25223644|2019-10-01 00:04:34|2019-10-01 00:35:42|             210|Ashland Ave & Div...|  

In [ ]:
def output_average_trip_duration(df, output_file_path):
  df_new = df.withColumn('start_time_ts', unix_timestamp('started_at'))\
                      .withColumn('end_time_ts', unix_timestamp('ended_at'))
  df_new = df_new.withColumn('days_diff', col('end_time_ts').cast('long') - col('start_time_ts').cast('long'))
  df_new = df_new.withColumn("day", to_date(col("started_at")))
  df_new = df_new.groupBy("day").agg(mean("days_diff").alias("trip_duration"))
  # select only day and average_trip_duration columns
  df_new = df_new.orderBy("day")
  df_new = df_new.dropna()
  write_to_csv(df_new, output_file_path)


In [ ]:
output_average_trip_duration(sparkDF, "/content/drive/MyDrive/reports/report1")
#

In [ ]:
def output_trips_per_day(df, output_file_path):
  df_new = df.withColumn("day", to_date(col("started_at")))
  df_new = df_new.groupBy("day").count()
  df_new = df_new.orderBy("day")
  write_to_csv(df_new, output_file_path)

In [ ]:
output_trips_per_day(sparkDF, "/content/drive/MyDrive/reports/report2")

In [ ]:
def output_most_popular_start_stations_for_each_month(df, output_file_path):
  df_new = df.withColumn("month", month(col("started_at")))
  df_new = df_new.dropna(subset=["start_station_id"])
  df_new = df_new.groupBy("month", "year", "start_station_id", "start_station_name").count()
  df_new = df_new.orderBy("count", "month", "year", ascending=[False, True, True])
  df_new.show()
  write_to_csv(df_new, output_file_path)

In [ ]:
output_most_popular_start_stations_for_each_month(sparkDF, "/content/drive/MyDrive/reports/report3")

+-----+----+----------------+--------------------+-----+
|month|year|start_station_id|  start_station_name|count|
+-----+----+----------------+--------------------+-----+
|   10|2019|             192| Canal St & Adams St| 6564|
|   10|2019|              77|Clinton St & Madi...| 5340|
|   10|2019|              91|Clinton St & Wash...| 4921|
|   10|2019|              35|Streeter Dr & Gra...| 4553|
|   10|2019|              76|Lake Shore Dr & M...| 3981|
|   10|2019|             195|Columbus Dr & Ran...| 3925|
|   10|2019|             287|Franklin St & Mon...| 3741|
|   10|2019|             133|Kingsbury St & Ki...| 3497|
|   11|2019|             192| Canal St & Adams St| 3445|
|   10|2019|              81|  Daley Center Plaza| 3387|
|   10|2019|              43|Michigan Ave & Wa...| 3294|
|    1|2020|             192| Canal St & Adams St| 3241|
|    3|2020|             675|               HQ QR| 3210|
|   12|2019|             192| Canal St & Adams St| 2928|
|    1|2020|              77|Cl

In [ ]:
def output_top_three_stations_each_day_for_last_two_weeks(df, output_file_path):
  df_new = df.withColumn("day", to_date(col("started_at")))
  df_new = df_new.withColumn("week", date_format(to_date("day", "yyyy-mm-dddd"), "W"))
  df_new = df_new.filter((col("week") == "3") | (col("week") == "4"))
  df_new = df_new.withColumn("station_usage", array(col("start_station_name"), col("end_station_name")))
  df_new = df_new.select("day", explode("station_usage").alias("station_name"))
  df_new = df_new.groupBy("day", "station_name").count()
  df_new = df_new.orderBy("day", "count", ascending=[True, False])
  window = Window.partitionBy("day").orderBy(col("count").desc())
  df_new = df_new.withColumn("rank", rank().over(window))
  df_new = df_new.filter(col("rank") <= 3)
  df_new = df_new.select("day", "station_name", "count")
  write_to_csv(df_new, output_file_path)

In [ ]:
output_top_three_stations_each_day_for_last_two_weeks(sparkDF, "/content/drive/MyDrive/reports/report4")

In [16]:
df_2019.drop(columns=['tripduration'], inplace=True)

In [17]:

df_2019['started_at']= pd.to_datetime(df_2019['started_at'])
df_2019['ended_at'] = pd.to_datetime(df_2019['ended_at'])

In [18]:
df_2019.head()

,ride_id,started_at,ended_at,bikeid,start_station_id,start_station_name,end_station_id,end_station_name,usertype,gender,birthyear,year
0,25223640,2019-10-01 00:01:39,2019-10-01 00:17:20,2215,20,Sheffield Ave & Kingsbury St,309,Leavitt St & Armitage Ave,Subscriber,Male,1987,2019
1,25223641,2019-10-01 00:02:16,2019-10-01 00:06:34,6328,19,Throop (Loomis) St & Taylor St,241,Morgan St & Polk St,Subscriber,Male,1998,2019
2,25223642,2019-10-01 00:04:32,2019-10-01 00:18:43,3003,84,Milwaukee Ave & Grand Ave,199,Wabash Ave & Grand Ave,Subscriber,Female,1991,2019
3,25223643,2019-10-01 00:04:32,2019-10-01 00:43:43,3275,313,Lakeview Ave & Fullerton Pkwy,290,Kedzie Ave & Palmer Ct,Subscriber,Male,1990,2019
4,25223644,2019-10-01 00:04:34,2019-10-01 00:35:42,5294,210,Ashland Ave & Division St,382,Western Ave & Congress Pkwy,Subscriber,Male,1987,2019


In [22]:
spark.createDataFrame(df_2019).schema

StructType([StructField('ride_id', StringType(), True), StructField('started_at', TimestampType(), True), StructField('ended_at', TimestampType(), True), StructField('bikeid', LongType(), True), StructField('start_station_id', LongType(), True), StructField('start_station_name', StringType(), True), StructField('end_station_id', LongType(), True), StructField('end_station_name', StringType(), True), StructField('usertype', StringType(), True), StructField('gender', StringType(), True), StructField('birthyear', DoubleType(), True), StructField('year', LongType(), True)])

In [23]:
schema = types.StructType([
    types.StructField("ride_id", types.StringType(), True),
    types.StructField("started_at", types.TimestampType(), True),
    types.StructField("ended_at", types.TimestampType(), True),
     types.StructField("bikeid", types.LongType(), True),
    types.StructField("start_station_id", types.LongType(), True),
    types.StructField("start_station_name", types.StringType(), True),
    types.StructField("end_station_id", types.LongType(), True),
    types.StructField("end_station_name", types.StringType(), True),
    types.StructField('usertype', types.StringType(), True),
    types.StructField("gender", types.StringType(), True),
    types.StructField("birthyear", types.LongType(), True),
    types.StructField("year", types.LongType(), True),

])

In [24]:
# Convert problematic columns to numeric types before creating the Spark DataFrame
for col_name in ['bikeid', 'start_station_id', 'end_station_id', 'year']:
    # Convert to numeric and handle NaNs by filling with -1 (or another appropriate value)
    df_2019[col_name] = pd.to_numeric(df_2019[col_name], errors='coerce').fillna(-1).astype(int)

In [ ]:
# drop rows where gender is nan
#df_2019 = df_2019.dropna(subset=['gender'])

In [25]:
# drop rows where birthyear is nan
df_2019 = df_2019.dropna(subset=['birthyear'])

In [26]:
spark_2019_df = spark.createDataFrame(df_2019, schema = schema)

In [27]:
spark_2019_df.show()

+--------+-------------------+-------------------+------+----------------+--------------------+--------------+--------------------+----------+------+---------+----+
| ride_id|         started_at|           ended_at|bikeid|start_station_id|  start_station_name|end_station_id|    end_station_name|  usertype|gender|birthyear|year|
+--------+-------------------+-------------------+------+----------------+--------------------+--------------+--------------------+----------+------+---------+----+
|25223640|2019-10-01 00:01:39|2019-10-01 00:17:20|  2215|              20|Sheffield Ave & K...|           309|Leavitt St & Armi...|Subscriber|  Male|     1987|2019|
|25223641|2019-10-01 00:02:16|2019-10-01 00:06:34|  6328|              19|Throop (Loomis) S...|           241| Morgan St & Polk St|Subscriber|  Male|     1998|2019|
|25223642|2019-10-01 00:04:32|2019-10-01 00:18:43|  3003|              84|Milwaukee Ave & G...|           199|Wabash Ave & Gran...|Subscriber|Female|     1991|2019|
|25223643|

In [ ]:
def output_average_trip_duration_by_gender(df, output_file_path):
  df_new = df.withColumn('start_time_ts', unix_timestamp('started_at'))\
                      .withColumn('end_time_ts', unix_timestamp('ended_at'))
  df_new = df_new.withColumn('days_diff', col('end_time_ts').cast('long') - col('start_time_ts').cast('long'))
  df_new = df_new.withColumn("day", to_date(col("started_at")))
  df_new = df_new.groupBy("gender").agg(mean("days_diff").alias("trip_duration"))
  write_to_csv(df_new, output_file_path)


In [ ]:
output_average_trip_duration_by_gender(spark_2019_df, "/content/drive/MyDrive/reports/report5")

In [49]:
def output_top_ten_ages_for_longest_trips(df, output_file_path):
  df_new = df.withColumn('start_time_ts', unix_timestamp('started_at'))\
                      .withColumn('end_time_ts', unix_timestamp('ended_at'))
  df_new = df_new.withColumn('days_diff', col('end_time_ts').cast('long') - col('start_time_ts').cast('long'))
  df_new = df_new.withColumn("day", to_date(col("started_at")))
  df_new = df_new.withColumn('age', (year(col('started_at')) - col('birthyear')).cast('integer'))
  window_spec = Window.partitionBy("age").orderBy(col("days_diff").desc())
  longest_trips = df_new.withColumn("rank", row_number().over(window_spec)) \
    .filter(col("rank") == 1) \
    .select("age", "days_diff") \
    .orderBy(col("days_diff").desc())

  # select only the top 10 rows
  longest_trips = longest_trips.limit(10)

  write_to_csv(longest_trips, output_file_path)


In [51]:
output_top_ten_ages_for_longest_trips(spark_2019_df, "/content/drive/MyDrive/reports/report6")

In [42]:
def output_top_ten_ages_for_shortest_trips(df, output_file_path):
  df_new = df.withColumn('start_time_ts', unix_timestamp('started_at'))\
                      .withColumn('end_time_ts', unix_timestamp('ended_at'))
  df_new = df_new.withColumn('days_diff', col('end_time_ts').cast('long') - col('start_time_ts').cast('long'))
  df_new = df_new.withColumn("day", to_date(col("started_at")))
  df_new = df_new.withColumn('age', (year(col('started_at')) - col('birthyear')).cast('integer'))
  window_spec = Window.partitionBy("age").orderBy(col("days_diff").asc())
  shortest_trips = df_new.withColumn("rank", row_number().over(window_spec)) \
    .filter(col("rank") == 1) \
    .select("age", "days_diff") \
    .orderBy(col("days_diff").asc())
  shortest_trips = shortest_trips.limit(10)
  write_to_csv(shortest_trips, output_file_path)


In [52]:
output_top_ten_ages_for_shortest_trips(spark_2019_df, "/content/drive/MyDrive/reports/report7")